# Customer Matching Example
## Matching Names
This notebook demonstrates how we use the Levenstein distance to match customer names: [Levenstein distance](https://en.wikipedia.org/wiki/Levenshtein_distance) to match. The Levenstein distance is a measure of the similarity between two strings. The distance is the number of deletions, insertions, or substitutions required to transform one string into the other. The distance is 0 if the two strings are identical.

In [85]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import pandas as pd
import numpy as np

# Import tqdm for progress bar
from tqdm import tqdm

In [86]:
customers = pd.read_parquet('../data/customers.parquet')
# Add a row
matchin_record_1 = {"id": "e5413bcb-2a9f-4c03-9261-cfc6349d9999",
                  "source": "facebook",
                  "firstname": "Jan",
                  "lastname": "Janssens",
                  "street": "Kerkstraat",
                  "housenumber": 1,
                  "box": "",
                  "postalcode": "1000",
                  "city": "Brussel",
                  "country": "Belgium",
                  "birthdate": "1980-01-01",
                  "email": "test_record@lol.com"}

matchin_record_2 = {"id": "e5413bcb-2a9f-4c03-9261-cfc6349d9999",
                  "source": "facebook",
                  "firstname": "Jan",
                  "lastname": "Janssens",
                  "street": "Kerkstraat",
                  "housenumber": 1,
                  "box": "",
                  "postalcode": "2000",
                  "city": "Antwerpen",
                  "country": "Belgium",
                  "birthdate": "1980-01-01",
                  "email": "test_record@lol.com"}

customers = customers.append([matchin_record_1, matchin_record_2], ignore_index=True)
customers.head(5)

/tmp/ipykernel_607/1367721769.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  customers = customers.append([matchin_record_1, matchin_record_2], ignore_index=True)


,id,source,firstname,lastname,street,housenumber,box,postalcode,city,country,birthdate,email
0,5ec33e41-d6db-4f86-8c1d-4ad694f7b3b8,twitter,Sylvie,De Maeyer,Vilstraat,60,,3511,Hasselt,Belgium,1970-09-10,janssensfrancois@example.com
1,05549698-6394-43b3-9633-0ef6fc46c1e5,facebook,Danny,Brouwers,Vossekotstraat,10,,9100,Sint-Niklaas,Belgium,1953-01-05,larsfrancois@example.com
2,f1cbe4b0-9bcf-46ec-8e3c-92632f9925e4,twitter,Johan,Leemans,Alfons De Cockstraat,12,0209,9310,Aalst,Belgium,1946-08-05,jonasde-winter@example.net
3,e5413bcb-2a9f-4c03-9261-cfc6349d9999,facebook,Jan,Janssens,Kerkstraat,1,,1000,Brussel,Belgium,1980-01-01,test_record@lol.com
4,e5413bcb-2a9f-4c03-9261-cfc6349d9999,facebook,Jan,Janssens,Kerkstraat,1,,2000,Antwerpen,Belgium,1980-01-01,test_record@lol.com


In [87]:
customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           5 non-null      object
 1   source       5 non-null      object
 2   firstname    5 non-null      object
 3   lastname     5 non-null      object
 4   street       5 non-null      object
 5   housenumber  5 non-null      object
 6   box          5 non-null      object
 7   postalcode   5 non-null      object
 8   city         5 non-null      object
 9   country      5 non-null      object
 10  birthdate    5 non-null      object
 11  email        5 non-null      object
dtypes: object(12)
memory usage: 608.0+ bytes


## Matching Customer Names
Now we will use the fuzzywuzzy library to match customer names. The fuzzywuzzy library is a python library that uses the Levenstein distance to match strings. 

In [88]:
# Here, fuzz.token_sort_ratio is one of the string matching algorithms provided by the fuzzywuzzy library .
# It calculates the similarity score between two strings based on the similarity of their tokens (words).
# The score is a number between 0 and 100, where 100 is a perfect match.

def similarity_score(record1, record2):
    return fuzz.token_sort_ratio(record1, record2)

In [89]:
# Now let's create a matching string that concatenates the first name, last name, and address of each customer.

customers['matching_string'] = customers['firstname'] + ' ' + customers['lastname'] + ' ' + customers['street'] + ' ' + customers['housenumber'].astype('str') +  ' ' +customers['postalcode'].astype('str')
customers['matching_string']

0            Sylvie De Maeyer Vilstraat 60 3511
1         Danny Brouwers Vossekotstraat 10 9100
2    Johan Leemans Alfons De Cockstraat 12 9310
3                Jan Janssens Kerkstraat 1 1000
4                Jan Janssens Kerkstraat 1 2000
Name: matching_string, dtype: object

In [90]:
matches_df = pd.DataFrame(columns=['record_id', 'record', 'match_id', 'match', 'score'])

for index, record in tqdm(customers.iterrows()):
    # Compare the matching string of the current record with the matching strings of all other records
    matches = process.extract(record['matching_string'], customers.drop(index)['matching_string'].to_list(), scorer=similarity_score)
    for match in matches:
        # Only add matches with a score of 90 or higher
        if match[1] >= 90:
            match_index = customers[customers['matching_string'] == match[0]].index[0]
            matches_df = pd.concat([matches_df, pd.DataFrame({'record_id': index, 'record': record['matching_string'], 'match_id': match_index, 'match': match[0], 'score': match[1]}, index=[0])], ignore_index=True)


5it [00:00, 1069.43it/s]


In [91]:
matches_df

,record_id,record,match_id,match,score
0,3,Jan Janssens Kerkstraat 1 1000,4,Jan Janssens Kerkstraat 1 2000,97
1,4,Jan Janssens Kerkstraat 1 2000,3,Jan Janssens Kerkstraat 1 1000,97
